In [317]:
import pandas as pd

In [318]:
import gzip
path = '/Users/marcushimelhoch/Downloads/'

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Grocery_and_Gourmet_Food_5.json.gz')

In [319]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1VEELTKS8NLZB,616719923X,Amazon Customer,"[0, 0]",Just another flavor of Kit Kat but the taste i...,4.0,Good Taste,1370044800,"06 1, 2013"
1,A14R9XMZVJ6INB,616719923X,amf0001,"[0, 1]",I bought this on impulse and it comes from Jap...,3.0,"3.5 stars, sadly not as wonderful as I had hoped",1400457600,"05 19, 2014"
2,A27IQHDZFQFNGG,616719923X,Caitlin,"[3, 4]",Really good. Great gift for any fan of green t...,4.0,Yum!,1381190400,"10 8, 2013"
3,A31QY5TASILE89,616719923X,DebraDownSth,"[0, 0]","I had never had it before, was curious to see ...",5.0,Unexpected flavor meld,1369008000,"05 20, 2013"
4,A2LWK003FFMCI5,616719923X,Diana X.,"[1, 2]",I've been looking forward to trying these afte...,4.0,"Not a very strong tea flavor, but still yummy ...",1369526400,"05 26, 2013"


In [320]:
df= df.drop(columns = [ 'reviewerName','helpful', 'unixReviewTime', 'reviewTime'])

In [370]:
df = df.drop(columns = ['reviewText','summary'])

In [371]:
df.head()

,reviewerID,asin,overall
0,A1VEELTKS8NLZB,616719923X,4.0
1,A14R9XMZVJ6INB,616719923X,3.0
2,A27IQHDZFQFNGG,616719923X,4.0
3,A31QY5TASILE89,616719923X,5.0
4,A2LWK003FFMCI5,616719923X,4.0


In [373]:
n_users = df.reviewerID.unique().shape[0]

In [377]:
import numpy as np

In [374]:
n_items = df.asin.unique().shape[0]

In [380]:
n_items

8713

In [381]:
data_matrix = np.zeros((n_users, n_items))

In [385]:
df.itertuples

<bound method DataFrame.itertuples of             reviewerID        asin  overall
0       A1VEELTKS8NLZB  616719923X      4.0
1       A14R9XMZVJ6INB  616719923X      3.0
2       A27IQHDZFQFNGG  616719923X      4.0
3       A31QY5TASILE89  616719923X      5.0
4       A2LWK003FFMCI5  616719923X      4.0
...                ...         ...      ...
151249  A2L6QS8SVHT9RG  B00KCJRVO2      4.0
151250   AFJFXN42RZ3G2  B00KCJRVO2      4.0
151251   ASEBX8TBYWQWA  B00KCJRVO2      5.0
151252   ANKQGTXHREOI5  B00KCJRVO2      4.0
151253  A2CF66KIQ3RKX3  B00KCJRVO2      4.0

[151254 rows x 3 columns]>

# Popularity & CF model using Turicreate

In [386]:
import turicreate
tu_data = turicreate.SFrame(df)

In [387]:
popularity_model = turicreate.popularity_recommender.create(tu_data, user_id = 'reviewerID', item_id = 'asin', target = 'overall')

Preparing data set.

Data has 151254 observations with 14681 users and 8713 items.

Data prepared in: 0.384025s

151254 observations to process; with 8713 unique items.

In [388]:
popularity_recomm = popularity_model.recommend(users = [1,2,3,4,5], k = 5)
popularity_recomm.print_rows(num_rows = 25)

+------------+------------+-------+------+
| reviewerID |    asin    | score | rank |
+------------+------------+-------+------+
|     1      | B0000CNU15 |  5.0  |  1   |
|     1      | B0000CFLIL |  5.0  |  2   |
|     1      | B0000CFLCT |  5.0  |  3   |
|     1      | B0000CDBQN |  5.0  |  4   |
|     1      | B00005C2M2 |  5.0  |  5   |
|     2      | B0000CNU15 |  5.0  |  1   |
|     2      | B0000CFLIL |  5.0  |  2   |
|     2      | B0000CFLCT |  5.0  |  3   |
|     2      | B0000CDBQN |  5.0  |  4   |
|     2      | B00005C2M2 |  5.0  |  5   |
|     3      | B0000CNU15 |  5.0  |  1   |
|     3      | B0000CFLIL |  5.0  |  2   |
|     3      | B0000CFLCT |  5.0  |  3   |
|     3      | B0000CDBQN |  5.0  |  4   |
|     3      | B00005C2M2 |  5.0  |  5   |
|     4      | B0000CNU15 |  5.0  |  1   |
|     4      | B0000CFLIL |  5.0  |  2   |
|     4      | B0000CFLCT |  5.0  |  3   |
|     4      | B0000CDBQN |  5.0  |  4   |
|     4      | B00005C2M2 |  5.0  |  5   |
|     5    

In [390]:
item_sim_model = turicreate.item_similarity_recommender.create(tu_data, user_id = 'reviewerID', item_id = 'asin', target = 'overall', similarity_type = 'cosine')

Preparing data set.

Data has 151254 observations with 14681 users and 8713 items.

Data prepared in: 0.328294s

Training model from provided data.

Gathering per-item and per-user statistics.

...

| 1.13s                               | 100              | 8713            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 2.19751s

In [391]:
item_sim_recomm = item_sim_model.recommend(users = [1,2,3,4,5], k = 5)
item_sim_recomm.print_rows(num_rows = 25)

+------------+------------+---------------------+------+
| reviewerID |    asin    |        score        | rank |
+------------+------------+---------------------+------+
|     1      | B002IEVJRY | 0.12219961762428283 |  1   |
|     1      | B00934WBRO | 0.11122578859329224 |  2   |
|     1      | B006MONQMC |  0.1016530978679657 |  3   |
|     1      | B0041NYV8E | 0.10069480538368225 |  4   |
|     1      | B005HG9ERW | 0.09588055968284608 |  5   |
|     2      | B002IEVJRY | 0.12219961762428283 |  1   |
|     2      | B00934WBRO | 0.11122578859329224 |  2   |
|     2      | B006MONQMC |  0.1016530978679657 |  3   |
|     2      | B0041NYV8E | 0.10069480538368225 |  4   |
|     2      | B005HG9ERW | 0.09588055968284608 |  5   |
|     3      | B002IEVJRY | 0.12219961762428283 |  1   |
|     3      | B00934WBRO | 0.11122578859329224 |  2   |
|     3      | B006MONQMC |  0.1016530978679657 |  3   |
|     3      | B0041NYV8E | 0.10069480538368225 |  4   |
|     3      | B005HG9ERW | 0.0

In [392]:
df.head()

,reviewerID,asin,overall
0,A1VEELTKS8NLZB,616719923X,4.0
1,A14R9XMZVJ6INB,616719923X,3.0
2,A27IQHDZFQFNGG,616719923X,4.0
3,A31QY5TASILE89,616719923X,5.0
4,A2LWK003FFMCI5,616719923X,4.0


# Matrix Factorization & Predicting User Ratings that are currently NaN

In [327]:
rating = pd.pivot_table(df, values = 'overall', index = ['reviewerID'], columns = ['asin'])

In [328]:
rating.sort_index(axis = 1, inplace = True)

In [393]:
rating.head()

asin,616719923X,9742356831,B00004S1C5,B0000531B7,B00005344V,B0000537AF,B00005C2M2,B00006IUTN,B0000CCZYY,B0000CD06J,...,B00IVT3LLW,B00IWBMCMS,B00J9IUCHA,B00JAXNMRG,B00JEL3N1E,B00JGPG60I,B00JL6LTMW,B00K00H9I6,B00KC0LGI8,B00KCJRVO2
reviewerID,,,,,,,,,,,,,,,,,,,,,
A00177463W0XWB16A9O05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A022899328A0QROR32DCT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A04309042SDSL8YX2HRR7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A068255029AHTHDXZURNU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A06944662TFWOKKV4GJKX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [398]:
#because the lowest rating is 1, i can fill all NaN values with 0 to represent no rating

In [394]:
ratings = rating.fillna(0)

In [395]:
ratings.head()

asin,616719923X,9742356831,B00004S1C5,B0000531B7,B00005344V,B0000537AF,B00005C2M2,B00006IUTN,B0000CCZYY,B0000CD06J,...,B00IVT3LLW,B00IWBMCMS,B00J9IUCHA,B00JAXNMRG,B00JEL3N1E,B00JGPG60I,B00JL6LTMW,B00K00H9I6,B00KC0LGI8,B00KCJRVO2
reviewerID,,,,,,,,,,,,,,,,,,,,,
A00177463W0XWB16A9O05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A022899328A0QROR32DCT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A04309042SDSL8YX2HRR7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A068255029AHTHDXZURNU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A06944662TFWOKKV4GJKX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [402]:
class MF():

    # Initializing the user-movie rating matrix, no. of latent features, alpha and beta.
    def __init__(self, R, K, alpha, beta, iterations):
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    # Initializing user-feature and movie-feature matrix 
    def train(self):
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initializing the bias terms
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])

        # List of training samples
        self.samples = [
        (i, j, self.R[i, j])
        for i in range(self.num_users)
        for j in range(self.num_items)
        if self.R[i, j] > 0
        ]

        # Stochastic gradient descent for given number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1) % 20 == 0:
                print("Iteration: %d ; error = %.4f" % (i+1, mse))

        return training_process

    # Computing total mean squared error
    def mse(self):
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    # Stochastic gradient descent to get optimized P and Q matrix
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    # Ratings for user i and moive j
    def get_rating(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    # Full user-movie rating matrix
    def full_matrix(self):
        return mf.b + mf.b_u[:,np.newaxis] + mf.b_i[np.newaxis:,] + mf.P.dot(mf.Q.T)

In [403]:
R = np.array(ratings)

In [407]:
mf = MF(R, K = 20, alpha=0.001, beta = 0.01, iterations = 100)
training_process = mf.train()
print()
print("P x Q:")
print(mf.full_matrix())
print()

Iteration: 20 ; error = 379.5982
Iteration: 40 ; error = 364.6512
Iteration: 60 ; error = 355.0153
Iteration: 80 ; error = 347.6986
Iteration: 100 ; error = 341.2795

P x Q:
[[4.19416718 4.68168383 4.27092143 ... 4.67948643 3.99989421 4.44777709]
 [3.48442614 4.02689956 3.6109085  ... 3.98100734 3.34345949 3.79389303]
 [3.86041338 4.4076995  4.02039345 ... 4.38876794 3.71067567 4.19191041]
 ...
 [4.36964002 4.8626674  4.4913279  ... 4.86629689 4.22978787 4.6842897 ]
 [4.50183423 5.00473647 4.65328508 ... 5.00375832 4.3171197  4.80932307]
 [3.83587924 4.34607603 3.95011198 ... 4.36916864 3.69791207 4.14307216]]



In [1]:
mf.full_matrix()

NameError: name 'mf' is not defined

In [344]:
from scipy import sparse

In [330]:
from lightfm import LightFM

/Users/marcushimelhoch/anaconda3/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [342]:
#create a sparse matrix from the data
copy_ratings = rating.copy()

In [351]:
matrix = copy_ratings.applymap(lambda x: 1 if x>2 else 0).as_matrix()

/Users/marcushimelhoch/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [356]:
matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [357]:
csr_matrix = sparse.csr_matrix(matrix)

In [362]:
csr_matrix.shape

(14681, 8713)

In [363]:
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

In [348]:
model = LightFM(no_components = 30, loss = 'warp')

In [358]:
#gradient descent, content + collaborative = hybrid

model.fit(csr_matrix)

In [364]:
precision = precision_at_k(model, csr_matrix, k = 10).mean()

In [366]:
auc = auc_score(model, csr_matrix).mean()

In [367]:
print('Precision: train %.2f' % (precision))
print('AUC: train %.2f' % (auc))

Precision: train 0.05
AUC: train 0.74


In [ ]:
predictions = model.predict()

In [360]:
movielens = fetch_movielens()

In [361]:
for key,value in movielens.items():
    print(key, type(value), value.shape)

train <class 'scipy.sparse.coo.coo_matrix'> (943, 1682)
test <class 'scipy.sparse.coo.coo_matrix'> (943, 1682)
item_features <class 'scipy.sparse.csr.csr_matrix'> (1682, 1682)
item_feature_labels <class 'numpy.ndarray'> (1682,)
item_labels <class 'numpy.ndarray'> (1682,)


In [268]:
users_no = rating.columns

In [299]:
categories = pd.DataFrame(meta.categories.value_counts().keys())

In [293]:
df_categories = pd.DataFrame(columns = categories)

In [294]:
df_users = pd.DataFrame(columns = df)

# Meta Data

In [104]:

meta = pd.read_csv('/Users/marcushimelhoch/Downloads/meta_filtered.csv')

In [105]:
meta.head()

,asin,salesRank,imUrl,categories,title,description,price,related,brand
0,616719923X,{'Grocery & Gourmet Food': 37305},http://ecx.images-amazon.com/images/I/51LdEao6...,[['Grocery & Gourmet Food']],Japanese Kit Kat Maccha Green Tea Flavor (5 Ba...,Green Tea Flavor Kit Kat have quickly become t...,NaN,"{'also_bought': ['B00FD63L5W', 'B0047YG5UY', '...",NaN
1,9742356831,{'Grocery & Gourmet Food': 3434},http://ecx.images-amazon.com/images/I/41pQp67A...,[['Grocery & Gourmet Food']],Mae Ploy Thai Green Curry Paste - 14 oz jar,Used to make various curry soups and stir fry ...,7.23,"{'also_bought': ['B000EI2LLO', 'B000EICISA', '...",Mae Ploy
2,B00004S1C5,{'Kitchen & Dining': 4494},http://ecx.images-amazon.com/images/I/41F75K9F...,"[['Grocery & Gourmet Food', 'Cooking & Baking'...","Ateco Food Coloring Kit, 6 colors","From Easter eggs to colorful cookies, Spectrum...",9.76,"{'also_bought': ['B0000CFMLT', 'B002PO3KBK', '...",HIC Harold Import Co.
3,B0000531B7,{'Grocery & Gourmet Food': 2858},http://ecx.images-amazon.com/images/I/519SuVj1...,[['Grocery & Gourmet Food']],"PowerBar Harvest Energy Bars, Double Chocolate...",NaN,24.75,"{'also_bought': ['B000EC63PU', 'B00DZGEY44', '...",Powerbar
4,B00005344V,{'Grocery & Gourmet Food': 5034},http://ecx.images-amazon.com/images/I/51H54cd-...,[['Grocery & Gourmet Food']],"Traditional Medicinals Breathe Easy, 16-Count ...","For nearly forty years, we&#x2019;ve been pass...",21.74,"{'also_bought': ['B0009F3POE', 'B0009F3POO', '...",Traditional Medicinals


In [106]:
meta['categories'] = meta['categories'].map(lambda x: x.lower().split('&'))

In [107]:
meta.head()

,asin,salesRank,imUrl,categories,title,description,price,related,brand
0,616719923X,{'Grocery & Gourmet Food': 37305},http://ecx.images-amazon.com/images/I/51LdEao6...,"[[['grocery , gourmet food']]]",Japanese Kit Kat Maccha Green Tea Flavor (5 Ba...,Green Tea Flavor Kit Kat have quickly become t...,NaN,"{'also_bought': ['B00FD63L5W', 'B0047YG5UY', '...",NaN
1,9742356831,{'Grocery & Gourmet Food': 3434},http://ecx.images-amazon.com/images/I/41pQp67A...,"[[['grocery , gourmet food']]]",Mae Ploy Thai Green Curry Paste - 14 oz jar,Used to make various curry soups and stir fry ...,7.23,"{'also_bought': ['B000EI2LLO', 'B000EICISA', '...",Mae Ploy
2,B00004S1C5,{'Kitchen & Dining': 4494},http://ecx.images-amazon.com/images/I/41F75K9F...,"[[['grocery , gourmet food', 'cooking , baki...","Ateco Food Coloring Kit, 6 colors","From Easter eggs to colorful cookies, Spectrum...",9.76,"{'also_bought': ['B0000CFMLT', 'B002PO3KBK', '...",HIC Harold Import Co.
3,B0000531B7,{'Grocery & Gourmet Food': 2858},http://ecx.images-amazon.com/images/I/519SuVj1...,"[[['grocery , gourmet food']]]","PowerBar Harvest Energy Bars, Double Chocolate...",NaN,24.75,"{'also_bought': ['B000EC63PU', 'B00DZGEY44', '...",Powerbar
4,B00005344V,{'Grocery & Gourmet Food': 5034},http://ecx.images-amazon.com/images/I/51H54cd-...,"[[['grocery , gourmet food']]]","Traditional Medicinals Breathe Easy, 16-Count ...","For nearly forty years, we&#x2019;ve been pass...",21.74,"{'also_bought': ['B0009F3POE', 'B0009F3POO', '...",Traditional Medicinals


In [108]:
meta.isnull().sum()

asin              0
salesRank       402
imUrl            13
categories        0
title            13
description    1204
price          1347
related         257
brand          2764
dtype: int64

In [110]:
meta = meta.dropna(subset = ['description'])

In [112]:
meta.head()

,asin,salesRank,imUrl,categories,title,description,price,related,brand
0,616719923X,{'Grocery & Gourmet Food': 37305},http://ecx.images-amazon.com/images/I/51LdEao6...,"[[['grocery , gourmet food']]]",Japanese Kit Kat Maccha Green Tea Flavor (5 Ba...,Green Tea Flavor Kit Kat have quickly become t...,NaN,"{'also_bought': ['B00FD63L5W', 'B0047YG5UY', '...",NaN
1,9742356831,{'Grocery & Gourmet Food': 3434},http://ecx.images-amazon.com/images/I/41pQp67A...,"[[['grocery , gourmet food']]]",Mae Ploy Thai Green Curry Paste - 14 oz jar,Used to make various curry soups and stir fry ...,7.23,"{'also_bought': ['B000EI2LLO', 'B000EICISA', '...",Mae Ploy
2,B00004S1C5,{'Kitchen & Dining': 4494},http://ecx.images-amazon.com/images/I/41F75K9F...,"[[['grocery , gourmet food', 'cooking , baki...","Ateco Food Coloring Kit, 6 colors","From Easter eggs to colorful cookies, Spectrum...",9.76,"{'also_bought': ['B0000CFMLT', 'B002PO3KBK', '...",HIC Harold Import Co.
4,B00005344V,{'Grocery & Gourmet Food': 5034},http://ecx.images-amazon.com/images/I/51H54cd-...,"[[['grocery , gourmet food']]]","Traditional Medicinals Breathe Easy, 16-Count ...","For nearly forty years, we&#x2019;ve been pass...",21.74,"{'also_bought': ['B0009F3POE', 'B0009F3POO', '...",Traditional Medicinals
5,B0000537AF,{'Health & Personal Care': 132146},http://ecx.images-amazon.com/images/I/41pmuVri...,"[[['grocery , gourmet food']]]","PowerBar ProteinPlus High Protein Bar, Vanilla...",The PowerBar ProteinPlus Protein Bar is a grea...,NaN,"{'also_bought': ['B001U89ITK', 'B009VV7G60', '...",NaN


In [113]:
meta.isnull().sum()

asin              0
salesRank       330
imUrl             0
categories        0
title             0
description       0
price          1079
related         190
brand          2323
dtype: int64

# Using Rake

In [192]:
text = meta.description.values

In [193]:
text

array(["Green Tea Flavor Kit Kat have quickly become the most sought after snack from Japan. Here are a limited supply of Maccha Green Tea flavored Kit Kat that you would not want to miss. These epic snacks have a sweet maccha green tea flavor mixed with creamy white chocolate, on a crispy wafer that Nestle's has perfected. Each bag contains 12 individually wrapped bars.",
       'Used to make various curry soups and stir fry dishes. Mae Ploy Brand is recognized in Thailand as a high quality export product with rich taste and authentic flavor. Packed in convenient plastic tub with tighly sealed lid. All natural.',
       "From Easter eggs to colorful cookies, Spectrum's gel food colorings provide a wide range of decorative touches, with gorgeous colors and an enormous range of depth. Each color--black, blue, yellow, green, pink, and red--comes in a soft-sided plastic squeeze bottle that allows you to add a drop of color at a time to frostings and fillings. Paste colors are much more in

In [196]:
r = Rake()

Unable to use extract_keywords_from_text on a full column

In [197]:
r.extract_keywords_from_text(text)

TypeError: cannot use a string pattern on a bytes-like object

when i try to perform in an iterating function, does not populate my new column with the information i want

In [204]:
meta['key_words'] = ""

for index, row in meta.iterrows():
    description = row['description']
    
    r = Rake()
    
    r.extract_keywords_from_text(description)
    
    key_words_dict_scores = r.get_word_degrees()
 
    row['key_words2'] = list(key_words_dict_scores.keys())

In [210]:
meta.head()

,asin,salesRank,imUrl,categories,title,description,price,related,brand,Key_words,key_words2,key_words
0,616719923X,{'Grocery & Gourmet Food': 37305},http://ecx.images-amazon.com/images/I/51LdEao6...,"[[['grocery , gourmet food']]]",Japanese Kit Kat Maccha Green Tea Flavor (5 Ba...,Green Tea Flavor Kit Kat have quickly become t...,NaN,"{'also_bought': ['B00FD63L5W', 'B0047YG5UY', '...",NaN,,,
1,9742356831,{'Grocery & Gourmet Food': 3434},http://ecx.images-amazon.com/images/I/41pQp67A...,"[[['grocery , gourmet food']]]",Mae Ploy Thai Green Curry Paste - 14 oz jar,Used to make various curry soups and stir fry ...,7.23,"{'also_bought': ['B000EI2LLO', 'B000EICISA', '...",Mae Ploy,,,
2,B00004S1C5,{'Kitchen & Dining': 4494},http://ecx.images-amazon.com/images/I/41F75K9F...,"[[['grocery , gourmet food', 'cooking , baki...","Ateco Food Coloring Kit, 6 colors","From Easter eggs to colorful cookies, Spectrum...",9.76,"{'also_bought': ['B0000CFMLT', 'B002PO3KBK', '...",HIC Harold Import Co.,,,
4,B00005344V,{'Grocery & Gourmet Food': 5034},http://ecx.images-amazon.com/images/I/51H54cd-...,"[[['grocery , gourmet food']]]","Traditional Medicinals Breathe Easy, 16-Count ...","For nearly forty years, we&#x2019;ve been pass...",21.74,"{'also_bought': ['B0009F3POE', 'B0009F3POO', '...",Traditional Medicinals,,,
5,B0000537AF,{'Health & Personal Care': 132146},http://ecx.images-amazon.com/images/I/41pmuVri...,"[[['grocery , gourmet food']]]","PowerBar ProteinPlus High Protein Bar, Vanilla...",The PowerBar ProteinPlus Protein Bar is a grea...,NaN,"{'also_bought': ['B001U89ITK', 'B009VV7G60', '...",NaN,,,


when i try on one string of text, it works

In [200]:
r.extract_keywords_from_text(text[0])

In [201]:
r.get_word_degrees()

defaultdict(<function rake_nltk.rake.Rake._build_word_co_occurance_graph.<locals>.<lambda>()>,
            {'snack': 1,
             'sweet': 6,
             'maccha': 12,
             'green': 17,
             'tea': 17,
             'flavor': 11,
             'mixed': 6,
             'want': 1,
             'would': 1,
             'limited': 2,
             'supply': 2,
             'miss': 1,
             'japan': 1,
             'crispy': 2,
             'wafer': 2,
             'perfected': 1,
             'bag': 6,
             'contains': 6,
             '12': 6,
             'individually': 6,
             'wrapped': 6,
             'bars': 6,
             'creamy': 3,
             'white': 3,
             'chocolate': 3,
             'nestle': 1,
             'flavored': 6,
             'kit': 11,
             'kat': 11,
             'quickly': 2,
             'become': 2,
             'sought': 1,
             'epic': 2,
             'snacks': 2})

# TFIDF , Truncated SVD and KMeans

In [224]:
meta.head()

,asin,salesRank,imUrl,categories,title,description,price,related,brand,Key_words,key_words2,key_words
0,616719923X,{'Grocery & Gourmet Food': 37305},http://ecx.images-amazon.com/images/I/51LdEao6...,"[[['grocery , gourmet food']]]",Japanese Kit Kat Maccha Green Tea Flavor (5 Ba...,Green Tea Flavor Kit Kat have quickly become t...,NaN,"{'also_bought': ['B00FD63L5W', 'B0047YG5UY', '...",NaN,,,
1,9742356831,{'Grocery & Gourmet Food': 3434},http://ecx.images-amazon.com/images/I/41pQp67A...,"[[['grocery , gourmet food']]]",Mae Ploy Thai Green Curry Paste - 14 oz jar,Used to make various curry soups and stir fry ...,7.23,"{'also_bought': ['B000EI2LLO', 'B000EICISA', '...",Mae Ploy,,,
2,B00004S1C5,{'Kitchen & Dining': 4494},http://ecx.images-amazon.com/images/I/41F75K9F...,"[[['grocery , gourmet food', 'cooking , baki...","Ateco Food Coloring Kit, 6 colors","From Easter eggs to colorful cookies, Spectrum...",9.76,"{'also_bought': ['B0000CFMLT', 'B002PO3KBK', '...",HIC Harold Import Co.,,,
4,B00005344V,{'Grocery & Gourmet Food': 5034},http://ecx.images-amazon.com/images/I/51H54cd-...,"[[['grocery , gourmet food']]]","Traditional Medicinals Breathe Easy, 16-Count ...","For nearly forty years, we&#x2019;ve been pass...",21.74,"{'also_bought': ['B0009F3POE', 'B0009F3POO', '...",Traditional Medicinals,,,
5,B0000537AF,{'Health & Personal Care': 132146},http://ecx.images-amazon.com/images/I/41pmuVri...,"[[['grocery , gourmet food']]]","PowerBar ProteinPlus High Protein Bar, Vanilla...",The PowerBar ProteinPlus Protein Bar is a grea...,NaN,"{'also_bought': ['B001U89ITK', 'B009VV7G60', '...",NaN,,,


In [225]:
text = meta.description.values

In [226]:
titles = meta.title.values

In [227]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()

csr_mat = tfidf.fit_transform(text)

print(csr_mat.toarray())

words = tfidf.get_feature_names()

print(words)


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
['00', '000', '000g', '000mg', '007810', '01', '0153', '0174', '02', '033617000187', '036192122848', '04', '041', '041508300360', '05', '0550', '05mg', '071998000013', '075779311107', '076808001464', '076808001471', '076808001785', '076808001792', '076808002515', '077330530057', '089836', '089836001917', '089836002624', '089836007810', '089836023056', '08lb', '08oz', '09', '094776078813', '094776078820', '096619181674item', '0cm', '0fat', '0g', '0gcholesterol', '0gnutritional', '0mg', '0mgprotein', '0oz', '0px', '0ysdarri', '0z', '10', '100', '1000', '1000s', '1005', '100c', '100ct', '100fat', '100g', '100mg', '100percent', '100th', '101', '1015', '102', '104', '1041', '105f', '107', '107g', '1090', '10g', '10mg', '10oz', '10px', '10x', '11', '110', '110288', '111', '111775', '112', '113', '1139', '113g', '114', '114f', '115', '116'

In [228]:
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline

In [252]:
#from sklearn.cluster import KMeans
#wcss = []

#for i in range(1, 11):
 #   kmeans = KMeans(n_clusters = i, init = 'k-means++',
                #    max_iter = 400, n_init = 10, random_state = 0)
  #  kmeans.fit(csr_mat)
   # wcss.append(kmeans.inertia_)
    
#Plotting the results onto a line graph to observe 'The elbow'
#plt.plot(range(1, 11), wcss)
#plt.title('Elbow Method')
#plt.xlabel('Association')
#plt.ylabel('WCSS') #within cluster sum of squares
#plt.show()

In [229]:
svd = TruncatedSVD(n_components = 50)

kmeans = KMeans(n_clusters = 15)

pipeline = make_pipeline(svd, kmeans)

In [230]:
pipeline.fit(csr_mat)

Pipeline(memory=None,
         steps=[('truncatedsvd',
                 TruncatedSVD(algorithm='randomized', n_components=50, n_iter=5,
                              random_state=None, tol=0.0)),
                ('kmeans',
                 KMeans(algorithm='auto', copy_x=True, init='k-means++',
                        max_iter=300, n_clusters=15, n_init=10, n_jobs=None,
                        precompute_distances='auto', random_state=None,
                        tol=0.0001, verbose=0))],
         verbose=False)

In [185]:
labels = pipeline.predict(csr_mat)

In [186]:
labels 

array([9, 2, 2, ..., 2, 2, 8], dtype=int32)

In [212]:
df = pd.DataFrame({'label': labels, 'item': titles})

In [213]:
df.sort_values('label')

,label,item
5328,0,Ghirardelli Peppermint Bark Squares with Dark ...
6871,0,Sanders Dark Chocolate Sea Salt Caramels 36 Ou...
4754,0,"Monin Flavored Sauce, Dark Chocolate, 12-Ounce..."
6765,0,"Ghirardelli Milk Chocolate Easter Eggs, 3.5-Ou..."
4753,0,"Nestle Ovaltine Rich Chocolate, 12-Ounce Tubs ..."
4401,0,"Cafe Escapes Dark Chocolate Hot Cocoa K-Cups, ..."
5269,0,"Green &amp; Black Organic Hot Chocolate Mix, 5..."
5268,0,Cadbury Easter Royal Dark Chocolate Candy Coat...
4191,0,Arnott's Tim Tam Original
3606,0,"Ghirardelli Triple Chocolate Brownie Mix, Semi..."


In [232]:
df.head()

,label,item
0,9,Japanese Kit Kat Maccha Green Tea Flavor (5 Ba...
1,2,Mae Ploy Thai Green Curry Paste - 14 oz jar
2,2,"Ateco Food Coloring Kit, 6 colors"
3,1,"Traditional Medicinals Breathe Easy, 16-Count ..."
4,2,"PowerBar ProteinPlus High Protein Bar, Vanilla..."


In [233]:
meta.head()

,asin,salesRank,imUrl,categories,title,description,price,related,brand,Key_words,key_words2,key_words
0,616719923X,{'Grocery & Gourmet Food': 37305},http://ecx.images-amazon.com/images/I/51LdEao6...,"[[['grocery , gourmet food']]]",Japanese Kit Kat Maccha Green Tea Flavor (5 Ba...,Green Tea Flavor Kit Kat have quickly become t...,NaN,"{'also_bought': ['B00FD63L5W', 'B0047YG5UY', '...",NaN,,,
1,9742356831,{'Grocery & Gourmet Food': 3434},http://ecx.images-amazon.com/images/I/41pQp67A...,"[[['grocery , gourmet food']]]",Mae Ploy Thai Green Curry Paste - 14 oz jar,Used to make various curry soups and stir fry ...,7.23,"{'also_bought': ['B000EI2LLO', 'B000EICISA', '...",Mae Ploy,,,
2,B00004S1C5,{'Kitchen & Dining': 4494},http://ecx.images-amazon.com/images/I/41F75K9F...,"[[['grocery , gourmet food', 'cooking , baki...","Ateco Food Coloring Kit, 6 colors","From Easter eggs to colorful cookies, Spectrum...",9.76,"{'also_bought': ['B0000CFMLT', 'B002PO3KBK', '...",HIC Harold Import Co.,,,
4,B00005344V,{'Grocery & Gourmet Food': 5034},http://ecx.images-amazon.com/images/I/51H54cd-...,"[[['grocery , gourmet food']]]","Traditional Medicinals Breathe Easy, 16-Count ...","For nearly forty years, we&#x2019;ve been pass...",21.74,"{'also_bought': ['B0009F3POE', 'B0009F3POO', '...",Traditional Medicinals,,,
5,B0000537AF,{'Health & Personal Care': 132146},http://ecx.images-amazon.com/images/I/41pmuVri...,"[[['grocery , gourmet food']]]","PowerBar ProteinPlus High Protein Bar, Vanilla...",The PowerBar ProteinPlus Protein Bar is a grea...,NaN,"{'also_bought': ['B001U89ITK', 'B009VV7G60', '...",NaN,,,


'Mae Ploy Thai Green Curry Paste - 14 oz jar'

# CountVectorizer with Description

In [234]:
#using a count matrix with item description

count = CountVectorizer()
count_matrix = count.fit_transform(meta['description'])

In [235]:
count_matrix.shape

(7509, 17251)

In [236]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.08084521, 0.17393131, ..., 0.11012975, 0.1713649 ,
        0.10102241],
       [0.08084521, 1.        , 0.21248509, ..., 0.28758185, 0.25831064,
        0.13884203],
       [0.17393131, 0.21248509, 1.        , ..., 0.33468064, 0.29057907,
        0.23509295],
       ...,
       [0.11012975, 0.28758185, 0.33468064, ..., 1.        , 0.41397466,
        0.29158274],
       [0.1713649 , 0.25831064, 0.29057907, ..., 0.41397466, 1.        ,
        0.32831724],
       [0.10102241, 0.13884203, 0.23509295, ..., 0.29158274, 0.32831724,
        1.        ]])

In [237]:
indices = pd.Series(meta['title'])

In [238]:
def recommendations(title, cosine_sim = cosine_sim):
    recommended_product = []
    
    idx = indices[indices == title].index[0]
    
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    for i in top_10_indexes:
        recommended_product.append(list(df.index)[i])
        
    return recommended_product

In [242]:
meta.title[1]

'Mae Ploy Thai Green Curry Paste - 14 oz jar'

In [241]:
recommendations('Mae Ploy Thai Green Curry Paste - 14 oz jar')

[723, 722, 724, 725, 3314, 2307, 1676, 5031, 4850, 6368]

In [247]:
meta.iloc[722].title

'Mae Ploy Thai Red Curry Paste - 14 ounce per jar'

In [248]:
meta.iloc[723].title

'Mae Ploy Thai Panang Curry Paste - 14 oz jar'

In [249]:
meta.iloc[725].title

'Mae Ploy Thai Yellow Curry Paste - 14 oz jar'

In [250]:
meta.iloc[5031].title

"Libby's Lima Beans, 15-Ounce Cans (Pack of 12)"

# Rake

In [314]:
meta.head()

,asin,salesRank,imUrl,categories,title,description,price,related,brand,Key_words,key_words2,key_words
0,616719923X,{'Grocery & Gourmet Food': 37305},http://ecx.images-amazon.com/images/I/51LdEao6...,"[[['grocery , gourmet food']]]",Japanese Kit Kat Maccha Green Tea Flavor (5 Ba...,Green Tea Flavor Kit Kat have quickly become t...,NaN,"{'also_bought': ['B00FD63L5W', 'B0047YG5UY', '...",NaN,,,
1,9742356831,{'Grocery & Gourmet Food': 3434},http://ecx.images-amazon.com/images/I/41pQp67A...,"[[['grocery , gourmet food']]]",Mae Ploy Thai Green Curry Paste - 14 oz jar,Used to make various curry soups and stir fry ...,7.23,"{'also_bought': ['B000EI2LLO', 'B000EICISA', '...",Mae Ploy,,,
2,B00004S1C5,{'Kitchen & Dining': 4494},http://ecx.images-amazon.com/images/I/41F75K9F...,"[[['grocery , gourmet food', 'cooking , baki...","Ateco Food Coloring Kit, 6 colors","From Easter eggs to colorful cookies, Spectrum...",9.76,"{'also_bought': ['B0000CFMLT', 'B002PO3KBK', '...",HIC Harold Import Co.,,,
4,B00005344V,{'Grocery & Gourmet Food': 5034},http://ecx.images-amazon.com/images/I/51H54cd-...,"[[['grocery , gourmet food']]]","Traditional Medicinals Breathe Easy, 16-Count ...","For nearly forty years, we&#x2019;ve been pass...",21.74,"{'also_bought': ['B0009F3POE', 'B0009F3POO', '...",Traditional Medicinals,,,
5,B0000537AF,{'Health & Personal Care': 132146},http://ecx.images-amazon.com/images/I/41pmuVri...,"[[['grocery , gourmet food']]]","PowerBar ProteinPlus High Protein Bar, Vanilla...",The PowerBar ProteinPlus Protein Bar is a grea...,NaN,"{'also_bought': ['B001U89ITK', 'B009VV7G60', '...",NaN,,,


In [315]:
meta['key_words'] = ""

for index, row in meta.iterrows():
    description = row['description']
    
    r = Rake()
    
    r.extract_keywords_from_text(description)
    
    key_words_dict_scores = r.get_word_degrees()
 
    row['key_words2'] = list(key_words_dict_scores.keys())

In [316]:
meta.head()

,asin,salesRank,imUrl,categories,title,description,price,related,brand,Key_words,key_words2,key_words
0,616719923X,{'Grocery & Gourmet Food': 37305},http://ecx.images-amazon.com/images/I/51LdEao6...,"[[['grocery , gourmet food']]]",Japanese Kit Kat Maccha Green Tea Flavor (5 Ba...,Green Tea Flavor Kit Kat have quickly become t...,NaN,"{'also_bought': ['B00FD63L5W', 'B0047YG5UY', '...",NaN,,,
1,9742356831,{'Grocery & Gourmet Food': 3434},http://ecx.images-amazon.com/images/I/41pQp67A...,"[[['grocery , gourmet food']]]",Mae Ploy Thai Green Curry Paste - 14 oz jar,Used to make various curry soups and stir fry ...,7.23,"{'also_bought': ['B000EI2LLO', 'B000EICISA', '...",Mae Ploy,,,
2,B00004S1C5,{'Kitchen & Dining': 4494},http://ecx.images-amazon.com/images/I/41F75K9F...,"[[['grocery , gourmet food', 'cooking , baki...","Ateco Food Coloring Kit, 6 colors","From Easter eggs to colorful cookies, Spectrum...",9.76,"{'also_bought': ['B0000CFMLT', 'B002PO3KBK', '...",HIC Harold Import Co.,,,
4,B00005344V,{'Grocery & Gourmet Food': 5034},http://ecx.images-amazon.com/images/I/51H54cd-...,"[[['grocery , gourmet food']]]","Traditional Medicinals Breathe Easy, 16-Count ...","For nearly forty years, we&#x2019;ve been pass...",21.74,"{'also_bought': ['B0009F3POE', 'B0009F3POO', '...",Traditional Medicinals,,,
5,B0000537AF,{'Health & Personal Care': 132146},http://ecx.images-amazon.com/images/I/41pmuVri...,"[[['grocery , gourmet food']]]","PowerBar ProteinPlus High Protein Bar, Vanilla...",The PowerBar ProteinPlus Protein Bar is a grea...,NaN,"{'also_bought': ['B001U89ITK', 'B009VV7G60', '...",NaN,,,


In [256]:
key_words_dict_scores

defaultdict(<function rake_nltk.rake.Rake._build_word_co_occurance_graph.<locals>.<lambda>()>,
            {'value': 1,
             'artificial': 2,
             'flavors': 2,
             'love': 1,
             'maple': 3,
             'brown': 3,
             'sugar': 3,
             'chex': 15,
             'wholesome': 1,
             'gluten': 14,
             'variety': 1,
             'recipes': 1,
             'well': 6,
             'free': 13,
             'oatmeal': 14,
             'mixes': 6,
             'com': 1,
             'ways': 1,
             'gives': 5,
             'em': 1,
             'great': 2,
             'taste': 2,
             'goes': 3,
             'beyond': 3,
             'breakfast': 6,
             'check': 1,
             'freedom': 2,
             'like': 4,
             'simplicity': 1,
             'preservatives': 1,
             'warm': 1,
             'enjoy': 1,
             'colors': 1,
             'cereal': 3,
             'nutritious

In [ ]:
r.extract_keywords_from_text(text[0])

In [ ]:
r.get_word_degrees()